# LangGraph: Agent
By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents  
- Agents are systems that use `LLMs` as reasoning engines to determine which actions to take and the inputs necessary to perform the actions  
- After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish `Tool-calling`

In [15]:
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_tavily import TavilySearch
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [14]:
import getpass
import os

In [10]:
TEMPERATURE = 0.2
NUM_PREDICT = 256
MODEL = "gemma3:12b-it-qat"

model = init_chat_model(
    model=MODEL,
    temperature=TEMPERATURE,
    num_predict=NUM_PREDICT,
    use_gpu=True,
    model_provider='ollama',
    enable_tools=True,
)

In [4]:
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

## Creating Tool

In [5]:
tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

### Invoke directly with args

In [6]:
tool.invoke({"query": "What are the key takeaway from GE2025 rally in singapore?"})

{'query': 'What are the key takeaway from GE2025 rally in singapore?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'GE2025: Key quotes from PM Lawrence Wong at PAP lunchtime rally',
   'url': 'https://www.channelnewsasia.com/singapore/ge2025-lawrence-wong-pap-lunchtime-rally-key-quotes-5094671',
   'content': "GE2025: Key quotes from PM Lawrence Wong at PAP lunchtime rally - CNA In the first lunchtime rally of this General Election, Prime Minister Lawrence Wong spoke about the impact of voting for more opposition members, the decision to raise GST and the need for a strong team in uncertain times. “That’s not how elections work,” Mr Wong said in response to Workers’ Party (WP) chief Pritam Singh’s comments last Friday asking voters to compare his candidates to PAP’s backbenchers. GE2025: 'Clear mandate' will mean PAP government can speak for Singapore 'confidently', says PM Wong GE2025: Voting for opposition weakens PAP government in time of ser

### Invoke with ToolCall
We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [7]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "GE 2025 May 1st Rally Locations"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

{"query": "GE 2025 May 1st Rally Locations", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "GE2025: 11 election rallies to be held on May 1, final day of ...", "url": "https://www.straitstimes.com/singapore/politics/ge2025-11-election-rallies-to-be-held-on-may-1-final-day-of-campaigning", "content": "GE2025: 11 election rallies to be held on May 1, final day of c


### Invoke Within Agent
We can use our tools directly with an agent executor by **binding the tool** to the agent. This gives the agent the ability to dynamically set the available arguments to the Tavily search tool.

In [8]:
# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

agent = create_react_agent(model, [tavily_search_tool])

In [11]:

user_input = "What are the parties contesting in the sembawang GRC for GE2025 in Singapore."

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

What are the parties contesting in the sembawang GRC for GE2025 in Singapore.


ResponseError: registry.ollama.ai/library/gemma3:12b-it-qat does not support tools (status code: 400)